# Imports

In [64]:
# from google.colab import drive
# drive.mount('/content/drive')

In [65]:
# global imports
# import os
# from google.colab import drive


In [66]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import json
import re
import nltk
import string

In [67]:
! pip install contractions
!pip install -U spacy
!python -m spacy download en_core_web_lg

     ---------------------------------------- 0.0/587.7 MB ? eta -:--:--
     -------------------------------------- 0.0/587.7 MB 660.6 kB/s eta 0:14:50
     -------------------------------------- 0.1/587.7 MB 919.0 kB/s eta 0:10:40
     ---------------------------------------- 0.7/587.7 MB 5.2 MB/s eta 0:01:54
     ---------------------------------------- 1.4/587.7 MB 8.2 MB/s eta 0:01:12
     --------------------------------------- 2.3/587.7 MB 10.3 MB/s eta 0:00:57
     --------------------------------------- 3.2/587.7 MB 12.0 MB/s eta 0:00:49
     --------------------------------------- 3.8/587.7 MB 12.2 MB/s eta 0:00:48
     --------------------------------------- 4.7/587.7 MB 13.1 MB/s eta 0:00:45
     --------------------------------------- 5.5/587.7 MB 13.6 MB/s eta 0:00:43
     --------------------------------------- 6.5/587.7 MB 14.4 MB/s eta 0:00:41
      -------------------------------------- 7.5/587.7 MB 15.1 MB/s eta 0:00:39
      -------------------------------------- 8.

In [68]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anura\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anura\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\anura\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [69]:
import spacy
# import ktrain
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
import re
from sklearn.feature_extraction.text import CountVectorizer
#from ktrain.text import StandardTextPreprocessor
import contractions
from gensim import corpora
from gensim.models import LdaModel
from gensim.parsing.preprocessing import preprocess_string
# getting stop_words from nltk and initializing a lemmatizer
stop_words = set(nltk.corpus.stopwords.words('english'))
lemmatizer = nltk.stem.WordNetLemmatizer()
from sklearn.cluster import KMeans
import time
import pickle
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize


# Data Cleaning

In [70]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
# import cupy as cp
import pandas as pd
import xml.etree.ElementTree as ET
# from google.colab import drive
from sklearn.compose import make_column_transformer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [71]:
class ABSADataset(Dataset):
    def __init__(self, texts, aspects, labels, tokenizer, max_length=128):
        self.texts = texts
        self.aspects = aspects
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        aspect = self.aspects[idx]
        label = self.labels[idx]

        encoding = self.tokenizer(text, aspect,
                                  return_tensors='pt',
                                  max_length=self.max_length,
                                  truncation=True,
                                  padding='max_length')

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label)
        }

def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Convert to lowercase
    text = text.lower()

    # Tokenize text
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]

    # Join tokens back into string
    clean_text = ' '.join(filtered_tokens)

    return clean_text


def clean_aspect(aspect):
    # Convert to lowercase
    aspect = aspect.lower()

    # Remove punctuation
    aspect = aspect.translate(str.maketrans('', '', string.punctuation))

    return aspect

In [72]:
# load the dataset with labels

dataset_path = '/content/drive/MyDrive/Sem4/NLP/Final project/'
dataset_path = 'Restaurants_Train_v2.xml'
dataset = []

f = ET.parse(dataset_path)
for i in f.getroot():
  for j in i.find("aspectCategories"):
    dataset.append({
        "input_text": clean_text(i.find("text").text),
        "aspect": clean_aspect(j.attrib['category']),
        "polarity": j.attrib['polarity']
    })

bgw_vectorizer = CountVectorizer(binary=False, stop_words='english', ngram_range=(1,4))
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,1))
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

In [73]:
df = pd.DataFrame(train_dataset)

In [74]:
df.head()

,input_text,aspect,polarity
0,decor nice casual fine dining,ambience,conflict
1,find returning quite often,anecdotesmiscellaneous,positive
2,seating trying get maximum amount people resta...,ambience,negative
3,like took leftover chicken poured oil sprinkle...,food,negative
4,finest latin women youll see life hang jimmys,ambience,positive


In [75]:
df['input_text'][4]

'finest latin women youll see life hang jimmys'

# Getting aspect nouns



In [76]:
# Load spaCy model
nlp = spacy.load("en_core_web_lg")


# Define a function to extract nouns (aspects) from a review
def extract_nouns_from_review(review, nlp):
    # Process the review using spaCy pipeline
    doc = nlp(review)
    # Extract nouns from the review
    nouns = [token.text for token in doc if token.pos_ == "NOUN"]
    return list(set(nouns))

# Extract nouns (aspects) for each review and add them as a new column to the DataFrame
df['extracted_nouns'] = df['input_text'].apply(lambda review: extract_nouns_from_review(review, nlp))

In [77]:
df.head()

,input_text,aspect,polarity,extracted_nouns
0,decor nice casual fine dining,ambience,conflict,"[dining, decor]"
1,find returning quite often,anecdotesmiscellaneous,positive,[]
2,seating trying get maximum amount people resta...,ambience,negative,"[people, seating, neighbor, amount]"
3,like took leftover chicken poured oil sprinkle...,food,negative,"[sauce, powder, pepper, oil, chicken, red]"
4,finest latin women youll see life hang jimmys,ambience,positive,"[women, life]"


In [78]:
df.shape

(2970, 4)

In [79]:
df = df[~(df['input_text']=='')] #removing blank reviews

In [80]:
df = df[df['input_text'].apply(lambda x: len(nltk.word_tokenize(x)))>1] #removing single worded reviews

In [81]:
df.shape

(2904, 4)

In [82]:
df.to_csv(dataset_path+'semeval_cleaned.csv', index=False)

In [83]:
df=pd.read_csv(dataset_path+'semeval_cleaned.csv')

In [84]:
df.shape

(2904, 4)

# Getting aspect categories

In [85]:
import spacy
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
# Load spaCy model with GloVe vectors
#!python -m spacy download en_core_web_lg #run this for every new session
nlp = spacy.load("en_core_web_lg")

# Define aspects and their corresponding words
aspects = {
    'food': 'food delicious tasty delicious flavor meal cuisine chef portion seasoned authentic',
    'service': 'service staff friendly helpful attentive prompt',
    'ambience': 'ambience ambiance atmosphere decor decoration cozy elegant modern chic view clean dirty dusty',
    'price': 'price value affordable expensive budget cost cheap',
    # 'anecdotesmiscellaneous': 'location parking cleanliness presentation variety'

}


# Define a function to compute aspect embeddings
def compute_aspect_embeddings(aspects, nlp):
    aspect_embeddings = {}
    for aspect, words in aspects.items():
        # Tokenize and average the word vectors for each aspect
        tokens = nlp(words)
        aspect_vectors = [token.vector for token in tokens if token.has_vector]
        aspect_embeddings[aspect] = np.mean(aspect_vectors, axis=0)
    return aspect_embeddings

# Define a function to compute review embeddings
def compute_review_embedding(review, nlp):
    # Process the review using spaCy pipeline
    doc = nlp(review)
    sentence_embeddings = []
    for sent in doc.sents:
        # Compute sentence embedding by averaging word vectors
        sent_vector = np.mean([word.vector for word in sent if word.has_vector], axis=0)
        sentence_embeddings.append(sent_vector)
    # Replace new words with zeros and compute review embedding
    review_embedding = np.mean(sentence_embeddings, axis=0)
    return np.nan_to_num(review_embedding)


def compute_sentence_embedding(sentence, nlp):
    # Process the sentence using spaCy pipeline
    doc = nlp(sentence)
    # Compute sentence embedding by averaging word vectors
    sent_vector = np.mean([word.vector for word in doc if word.has_vector], axis=0)
    # Replace NaN values with zeros
    return np.nan_to_num(sent_vector)


# Compute semantic similarity between review sentences and aspect embeddings
def compute_semantic_similarity(review_sentences, aspect_embeddings):
    aspect_names = list(aspect_embeddings.keys())
    similarity_scores = []
    for sentence in review_sentences:
        sentence_vector = compute_review_embedding(sentence, nlp)
        sentence_vector = sentence_vector.reshape(1, -1)  # Reshape to match cosine_similarity function input
        sentence_similarity = []
        for aspect_name, aspect_vector in aspect_embeddings.items():
            aspect_vector = aspect_vector.reshape(1, -1)  # Reshape to match cosine_similarity function input
            similarity = cosine_similarity(sentence_vector, aspect_vector)[0][0]
            sentence_similarity.append(similarity)
        similarity_scores.append({aspect_name: score for aspect_name, score in zip(aspect_names, sentence_similarity)})
    return similarity_scores

In [86]:
# review_sentences = df['input_text'][0:100].to_list() --working fine

# review_sentences = df['input_text']

review_sentences = df['input_text']

#for each review in 'cleaned_text' and get a list of aspects

# Compute aspect embeddings
aspect_embeddings = compute_aspect_embeddings(aspects, nlp)

# Compute semantic similarity
semantic_similarity_scores = compute_semantic_similarity(review_sentences, aspect_embeddings)

# Choose the aspect with the highest similarity score for each sentence
chosen_aspects = [max(scores, key=scores.get) for scores in semantic_similarity_scores]

In [87]:
# Output the chosen aspects for each sentence
for sentence, aspect in zip(review_sentences, chosen_aspects):
    print("Sentence:", sentence)
    print("Chosen Aspect:", aspect)
    print()

Sentence: decor nice casual fine dining
Chosen Aspect: ambience

Sentence: find returning quite often
Chosen Aspect: service

Sentence: seating trying get maximum amount people restaurant nice neighbor dine
Chosen Aspect: service

Sentence: like took leftover chicken poured oil sprinkled pepper powder sauce translucent red
Chosen Aspect: food

Sentence: finest latin women youll see life hang jimmys
Chosen Aspect: ambience

Sentence: nice family owned traditional restaurant
Chosen Aspect: food

Sentence: far favorite place neighborhood
Chosen Aspect: ambience

Sentence: ive bunch times service always outstanding
Chosen Aspect: service

Sentence: food outstanding little perks great
Chosen Aspect: food

Sentence: terrible experience
Chosen Aspect: service

Sentence: cute place could good need get act together
Chosen Aspect: service

Sentence: worth visit
Chosen Aspect: price

Sentence: makhani ok korma bland
Chosen Aspect: food

Sentence: bravo blue hill
Chosen Aspect: ambience

Sentence:

In [88]:
df['aspect_predicted'] = chosen_aspects

In [89]:
df.head()

,input_text,aspect,polarity,extracted_nouns,aspect_predicted
0,decor nice casual fine dining,ambience,conflict,"['dining', 'decor']",ambience
1,find returning quite often,anecdotesmiscellaneous,positive,[],service
2,seating trying get maximum amount people resta...,ambience,negative,"['people', 'seating', 'neighbor', 'amount']",service
3,like took leftover chicken poured oil sprinkle...,food,negative,"['sauce', 'powder', 'pepper', 'oil', 'chicken'...",food
4,finest latin women youll see life hang jimmys,ambience,positive,"['women', 'life']",ambience


# Accuracy check

In [90]:
df2 = df[df['aspect'] != "anecdotesmiscellaneous"]

In [91]:
cnt = (df2['aspect'] == df2['aspect_predicted']).sum()

In [92]:
cnt

1318

In [93]:
df2.shape

(2033, 5)

In [94]:
1318/2033

0.6483030004918839

# 65% accuracy

In [129]:
import ngram_nb
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
train_idx = int(.8*len(df2))
model = ngram_nb.Ngram_NB(1)

shuffled_df = df2.sample(frac=1)
df_train = shuffled_df[:train_idx]
df_test = shuffled_df[train_idx:]
model.train(df_train, text_col = "input_text", category_col = "aspect")

df_polarity_predictions = []
for category in df_test["aspect"].unique():
    df_test_category = df_test[df_test["aspect"] == category]
    y_pred = df_test_category["input_text"].apply(lambda x: model.score(x, category = category)[0])
    y_true = df_test_category["polarity"]
    print(category, accuracy_score(y_true, y_pred))
    conf_mat = confusion_matrix(y_true, y_pred, labels = y_true.unique())
    print(y_true.unique())
    print(conf_mat)
    df_test_category["polarity_pred"] = y_pred
    df_polarity_predictions.append(df_test_category)
    
df_test_category
pd.concat(df_polarity_predictions).reset_index().to_csv("ngram_nb_predictions_df.csv")

price 0.76
['negative' 'positive' 'conflict' 'neutral']
[[16  6  0  0]
 [ 4 22  0  0]
 [ 0  1  0  0]
 [ 0  1  0  0]]
ambience 0.6575342465753424
['conflict' 'positive' 'negative' 'neutral']
[[ 0  9  3  0]
 [ 0 41  0  0]
 [ 0 12  7  0]
 [ 0  1  0  0]]


C:\Users\anura\AppData\Local\Temp\ipykernel_10968\1734844633.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_category["polarity_pred"] = y_pred
C:\Users\anura\AppData\Local\Temp\ipykernel_10968\1734844633.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_category["polarity_pred"] = y_pred


food 0.6946107784431138
['positive' 'neutral' 'conflict' 'negative']
[[111   0   0   4]
 [ 10   0   0   2]
 [ 12   0   0   2]
 [ 21   0   0   5]]
service 0.7008547008547008
['negative' 'positive' 'conflict' 'neutral']
[[29 14  0  0]
 [12 53  0  0]
 [ 1  5  0  0]
 [ 0  3  0  0]]


C:\Users\anura\AppData\Local\Temp\ipykernel_10968\1734844633.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_category["polarity_pred"] = y_pred
C:\Users\anura\AppData\Local\Temp\ipykernel_10968\1734844633.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_category["polarity_pred"] = y_pred
